# 모델 다운로드

In [ ]:
!pip install openvino-dev==2024.0.0

In [ ]:
!omz_downloader -h

In [ ]:
!omz_downloader --name face-detection-adas-0001 --precision FP16

** intel 폴더가 생성되고 그 안에 face-detection 폴더 안에 모델이 다운로드된다.

In [ ]:
import openvino as ov

In [ ]:
core=ov.Core()

model=core.read_model(model="models/face-detection-adas-0001.xml")
face_model=core.compile_model(model=model, device_name="CPU")

face_input_layer=face_model.input(0)
face_output_layer=face_model.output(0)
print("Input layer shape:", face_input_layer.shape)
print("Onput layer shape:", face_output_layer.shape)

## Prepare Input Data

In [ ]:
import cv2
import numpy as np

image=cv2.imread("data/test.jpg")  # 이미지를 불러온다.

image.shape                        # 원래이미지의 정보를 보자

In [ ]:
resized_image=cv2.resize(src=image, dsize=(672,384))  # input 사이즈로 조정->(3,672,384)
transposed_image=resized_image.transpose(2,0,1)       #Transpose후 (3,384,672)로 조정
input_image=np.expand_dims(transposed_image,0)        #차원확장 (1,3,384,672) 으로 조정

input_image.shape

## Inference(추론)

In [ ]:
face_output=face_model([input_image])[face_output_layer]

In [ ]:
face_output

In [ ]:
#[참고]넘파이 배열을 이쁘게 보이게 한다.
np.set_printoptions(threshold=np.inf, linewidth=np.inf)
print(face_output)

In [ ]:
# 원래 기본값으로 보이게 한다.
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=75, nanstr='nan', precision=8, suppress=False, threshold=1000, formatter=None)
face_output

## 후처리(이미지에 박스처리)

In [ ]:
#추론한 값(output), 이미지(image), 임계값(conf)을 가져와 
#이미지에 얼굴에 박스를 그리는 DrawBoundingBoxes함수
def DrawBoundingBoxes(output, image, conf):

    canvas = image.copy()
    h,w,_ = canvas.shape 

    predictions = output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]         # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]

    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택

    for detection in top_predictions:
        box = detection[3:7] * np.array([w, h, w, h]) # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box.astype("int")  # xmin, ymin, xmax, ymax에 상자 위치 값 지정

        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 만들기

    return canvas

In [ ]:
canvas = DrawBoundingBoxes(face_output, image, conf=0.5)

cv2.imshow("Canvas", canvas)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 배포

## Gradio 활용

### predict_image를 만들어야한다.

In [ ]:
import openvino as ov

core = ov.Core()
options=core.available_devices
options

In [ ]:
import openvino as ov
import cv2
import numpy as np

# 1. AI 모델 불러오기 ,input_layer & output_layer 준비
core = ov.Core()
model = core.read_model(model="models/face-detection-adas-0001.xml")
face_model = core.compile_model(model=model, device_name="CPU")
face_input_layer = face_model.input(0)
face_output_layer = face_model.output(0)

# 2. 새로 입력된 이미지 데이터 전처리
def preprocess(new_image):
    # Convert the PIL image to a NumPy array and resize to 224x224
    #image = np.array(new_image)  # Convert PIL image to numpy array
    resized_image = cv2.resize(src=new_image, dsize=(672, 384)) 
    transposed_image = resized_image.transpose(2, 0, 1)
    input_image = np.expand_dims(transposed_image, 0)
    return input_image

# 4. AI 추론 결과 후처리: 시각화(인식된 얼굴 주변에 박스 그리기)
def DrawBoundingBoxes(new_image, face_output, conf):
    canvas = new_image.copy()
    h,w,_ = canvas.shape 
    predictions = face_output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]         # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]
    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택
    for detection in top_predictions:
        box = detection[3:7] * np.array([w, h, w, h]) # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box.astype("int")  # xmin, ymin, xmax, ymax에 상자 위치 값 지정
        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 만들기
    return canvas

# 3. AI 추론
def predict_image(new_image):
    input_image = preprocess(new_image)  # Preprocess the image
    face_output = face_model([input_image])[face_output_layer]  # Perform inference  
    canvas = DrawBoundingBoxes(new_image, face_output, conf=0.5)
    return canvas

In [ ]:
import gradio as gr

# Gradio 실행
gr.Interface(fn=predict_image,
             inputs=gr.Image(type="numpy"),  # Use NumPy array
             outputs=gr.Image(type="numpy"),  # Output as NumPy array
             examples=["./data/test.jpg"]).launch()

## 영상추론

In [ ]:
import openvino as ov
import cv2
import numpy as np


# AI 모델 불러오기 ,input_layer & output_layer 준비
core = ov.Core()
model = core.read_model(model="models/face-detection-adas-0001.xml")
face_model = core.compile_model(model=model, device_name="CPU")
face_input_layer = face_model.input(0)
face_output_layer = face_model.output(0)

# 2. 새로 입력된 이미지 데이터 전처리
def preprocess(new_image):
    # Convert the PIL image to a NumPy array and resize to 224x224
    #image = np.array(new_image)  # Convert PIL image to numpy array
    resized_image = cv2.resize(src=new_image, dsize=(672, 384)) 
    transposed_image = resized_image.transpose(2, 0, 1)
    input_image = np.expand_dims(transposed_image, 0)
    return input_image

# AI 추론 결과 후처리: 시각화(인식된 얼굴 주변에 박스 그리기)
def DrawBoundingBoxes(new_image, face_output, conf):
    canvas = new_image.copy()
    h,w,_ = canvas.shape 
    predictions = face_output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]         # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]
    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택
    for detection in top_predictions:
        box = detection[3:7] * np.array([w, h, w, h]) # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box.astype("int")  # xmin, ymin, xmax, ymax에 상자 위치 값 지정
        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 만들기
    return canvas


# 3. AI 추론
def predict_image(new_image):
    input_image = preprocess(new_image)  # Preprocess the image
    face_output = face_model([input_image])[face_output_layer]  # Perform inference  
    canvas = DrawBoundingBoxes(new_image, face_output, conf=0.5)
    return canvas

### ->웹캠출력

In [4]:
# camera=cv2.VideoCapture(0)
# bg=cv2.imread("data/background.jpg")
camera=cv2.VideoCapture("data/video.mp4")
#camera = cv2.VideoCapture(0) #create a VideoCapture object with the 'first' camera (your webcam)

while(True):
    ret, frame = camera.read()             # Capture frame by frame      
    if ret == False:
        break
    # AI 추론을 위한 데이터 전처리
    resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
    transposed_frame = resized_frame.transpose(2, 0, 1)
    input_frame = np.expand_dims(transposed_frame, 0) 
    # AI 추론
    face_output = face_model([input_frame])[face_output_layer]
    # AI 추론 결과 시각화: 박스 그기기
    canvas = DrawBoundingBoxes(frame, face_output, conf=0.5)
    cv2.imshow('Press Spacebar to Exit', canvas)
    # 영상 종료: 스페이스 바
    if cv2.waitKey(1) & 0xFF == ord(' '):  # Stop if spacebar is detected
        break
camera.release()                           # Cleanup after spacebar is detected.
cv2.destroyAllWindows()